## Integrate the particles where they feel all NBody forces

In [1]:
import tstrippy
from gcs import path_handler as ph
import gcs
import h5py
import os 

In [12]:
montecarlokey = "monte-carlo-000"
internal_dynamics = "isotropic-plummer"
NP = int(1e5)

In [13]:
GCnames=list(tstrippy.Parsers.baumgardtMWGCs().data['Cluster'][:])

In [3]:
# now get the positions at the begining of the orbit 
potential="pouliasis2017pii-GCNBody"
orbit_file_name = ph.GC_orbits(potential,GCnames[0])

In [4]:
phase_space=gcs.extractors.GCOrbits.initial_simulation_coordinates_all(GCnames,potential,montecarlokey)

In [5]:
Masses,rh_mes,RAes,DECes,Rsunes,RVes,mualphaes,mu_deltaes=gcs.extractors.MonteCarloObservables.extract_all_GC_observables(GCnames,montecarlokey)

In [11]:
fname = ph.ParticleInitialConditions(GCnames[0])
x,y,z,vx,vy,vz=gcs.extractors.ParticleInitialConditions.load_particles(fname,internal_dynamics,montecarlokey,NP)